# Putting it all together

So far we have done the following on the prior Notebooks:

- **Notebook 01**: We loaded the Azure Search Engine with enriched PDFs in index: "cogsrch-index-files"
- **Notebook 02**: We loaded more information to the Search Engine this time using a CSV file with 90k rows/articles in index: "cogsrch-index-csv"
- **Notebook 03**: We added AzureOpenAI GPT models to enhance the the production of the answer by using Utility Chains of LLMs
- **Notebook 04**: We manually loaded an index with large/complex PDFs information , "cogsrch-index-books-vector"
- **Notebook 05**: We added memory to our system in order to power a conversational Chat Bot
- **Notebook 06**: We introduced Agents and Tools and built the first Skill/Agent, that can do RAG over a search engine
- **Notebook 07**: We build a second Agent (Pandas) in order to be able to solve a more complex task: ask questions to Tabular datasets
- **Notebook 08**: We used a SQL Agent in order to talk to a SQL Database directly
- **Notebook 09**: We used another  Agent in order to talk to the Bing Search API and create a Bing Chat Clone and implemented callbacks for real-time streaming and tool information
- **Notebook 10**: We built an API Agent that can translate a question into the right API calls, giving us the capability to talk to any datasource that provides a RESTFul API.


We are missing one more thing: **How do we glue all these features together into a very smart GPT Smart Search Engine Chat Bot?**

We want a virtual assistant for our company that can get the question, think what tool to use, then get the answer. The goal is that, regardless of the source of the information (Search Engine, Bing Search, SQL Database, CSV File, JSON File, APIs, etc), the Assistant can answer the question correctly using the right tool.

In this Notebook we are going to create that "brain" Agent (also called Master Agent), that:

1) understands the question, interacts with the user 
2) talks to other specialized Agents that are connected to diferent sources
3) once it get's the answer it delivers it to the user or let the specialized Agent to deliver it directly

This is the same concept of [AutoGen](https://www.microsoft.com/en-us/research/blog/autogen-enabling-next-generation-large-language-model-applications/): Agents talking to each other.

![image](https://www.microsoft.com/en-us/research/uploads/prod/2023/09/AutoGen_Fig1.png)

In [1]:
import os
import random
import json
import requests
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain.callbacks.manager import CallbackManager
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables import ConfigurableField

#custom libraries that we will use later in the app
from common.utils import (
    DocSearchAgent, 
    CSVTabularAgent, 
    SQLSearchAgent, 
    ChatGPTTool, 
    BingSearchAgent, 
    APISearchAgent, 
    reduce_openapi_spec
)
from common.callbacks import StdOutCallbackHandler
from common.prompts import CUSTOM_CHATBOT_PROMPT 

from dotenv import load_dotenv
load_dotenv("credentials.env")

from IPython.display import Markdown, HTML, display 

def printmd(string):
    display(Markdown(string))

MODEL_DEPLOYMENT_NAME = "gpt-35-turbo-1106" # Reminder: gpt-35-turbo models will create parsing errors and won't follow instructions correctly 

In [2]:
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

### Get the Tools - DocSearch Agent, CSV Agent, SQL Agent, Web Search Agent, ChatGPT, API Agent

**Consider the following concept:** Agents, which are essentially software entities designed to perform specific tasks, can be equipped with tools. These tools themselves can be other agents, each possessing their own set of tools. This creates a layered structure where tools can range from code sequences to human actions, forming interconnected chains. Ultimately, you're constructing a network of agents and their respective tools, all collaboratively working towards solving a specific task (This is what ChatGPT is). This network operates by leveraging the unique capabilities of each agent and tool, creating a dynamic and efficient system for task resolution.

In the file `common/utils.py` we created Agent Tools Classes for each of the Functionalities that we developed in prior Notebooks. 

In [3]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

COMPLETION_TOKENS = 2000

# We can run the everything with GPT3.5, but try also GPT4 and see the difference in the quality of responses
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=COMPLETION_TOKENS, streaming=True, callback_manager=cb_manager)

In [4]:
# DocSearchAgent is our Custom Tool Class (Agent) created for Azure Cognitive Search + OpenAI searches
text_indexes = ["cogsrch-index-files", "cogsrch-index-csv", "cogsrch-index-books"]
doc_search = DocSearchAgent(llm=llm, indexes=text_indexes,
                           k=6, reranker_th=1,
                           sas_token=os.environ['BLOB_SAS_TOKEN'],
                           callback_manager=cb_manager, verbose=False)

In [5]:
# BingSearchAgent is a langchain Tool class to use the Bing Search API (https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)
www_search = BingSearchAgent(llm=llm, k=5, callback_manager=cb_manager)

In [6]:
## CSVTabularAgent is a custom Tool class crated to Q&A over CSV files
file_url = "./data/all-states-history.csv"
csv_search = CSVTabularAgent(path=file_url, llm=llm, callback_manager=cb_manager)

In [7]:
## SQLDbAgent is a custom Tool class created to Q&A over a MS SQL Database
sql_search = SQLSearchAgent(llm=llm, k=30, callback_manager=cb_manager)

In [8]:
## ChatGPTTool is a custom Tool class created to talk to ChatGPT knowledge
chatgpt_search = ChatGPTTool(llm=llm, callback_manager=cb_manager)

In [10]:
## APISearchAgent is a custom Tool class created to talk to any API 

url = "https://datasetsgptsmartsearch.blob.core.windows.net/apispecs/openapi_kraken.json"
spec = requests.get(url + os.environ['BLOB_SAS_TOKEN']).json()

api_search = APISearchAgent(llm=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=1000),
                            llm_search=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=1000),
                            api_spec=str(reduce_openapi_spec(spec)),
                            callback_manager=cb_manager)

### Variables/knobs to use for customization

As you have seen so far, there are many knobs that you can dial up or down in order to change the behavior of your GPT Smart Search engine application, these are the variables you can tune:

- <u>llm</u>:
  - **deployment_name**: this is the deployment name of your Azure OpenAI model. This of course dictates the level of reasoning and the amount of tokens available for the conversation. For a production system you will need gpt-4-32k. This is the model that will give you enough reasoning power to work with agents, and enough tokens to work with detailed answers and conversation memory.
  - **temperature**: How creative you want your responses to be
  - **max_tokens**: How long you want your responses to be. It is recommended a minimum of 500
- <u>Tools</u>: To each tool you can add the following parameters to modify the defaults (set in utils.py), these are very important since they are part of the system prompt and determines what tool to use and when.
  - **name**: the name of the tool
  - **description**: when the brain agent should use this tool
- <u>DocSearchAgent</u>: 
  - **k**: The top k results per index from the text search action
  - **similarity_k**: top k results combined from the vector search action
  - **reranker_th**: threshold of the semantic search reranker. Picks results that are above the threshold. Max possible score=4
- <u>BingSearchAgent</u>:
  - **k**: The top k results from the bing search action
- <u>SQLSearchAgent</u>:
  - **k**: The top k results from the SQL search action. Adds TOP clause to the query
  
in `utils.py` you can also tune:
- <u>model_tokens_limit</u>: In this function you can edit what is the maximum allows of tokens reserve for the content. Remember that the remaining will be for the system prompt plus the answer

### Test the Tools

In [11]:
# Test the Documents Search Tool with a question we know it doesn't have the knowledge for
printmd(doc_search.run("what is the weather today in Dallas?"))

Tool: docsearch
Agent Action: 
Invoking: `docsearch` with `{'query': 'weather today in Dallas'}`



I couldn't find specific information about the weather today in Dallas in the retrieved documents. If you have access to the internet, you may want to check a reliable weather website or app for the most up-to-date weather forecast for Dallas.

I couldn't find specific information about the weather today in Dallas in the retrieved documents. If you have access to the internet, you may want to check a reliable weather website or app for the most up-to-date weather forecast for Dallas.

In [12]:
# Test the Document Search Tool with a question that we know it has the answer for
printmd(doc_search.run("How Covid affects obese people? and elderly?"))

Tool: docsearch
Agent Action: 
Invoking: `docsearch` with `{'query': 'Effects of COVID-19 on obese individuals'}`



Agent Action: 
Invoking: `docsearch` with `{'query': 'Effects of COVID-19 on elderly individuals'}`



### Effects of COVID-19 on Obese Individuals:
- **Risk Factor**: The WHO considers non-communicable diseases, such as obesity, a major risk factor for severe illness with COVID-19<sup><a href="https://doi.org/10.2337/dc20-0576" target="_blank">[1]</a></sup>.
- **Impact on Critical Care**: Studies show that a significant percentage of individuals in critical care units with serious COVID-19 complications are either overweight or obese, highlighting the impact of obesity on severe COVID-19 cases<sup><a href="https://doi.org/10.1002/oby.22867" target="_blank">[2]</a></sup>.
- **Severity Association**: Obese patients have increased odds of developing severe COVID-19, with higher risks compared to normal-weight individuals<sup><a href="https://doi.org/10.2337/dc20-0576" targ

### Effects of COVID-19 on Obese Individuals:
- **Risk Factor**: The WHO considers non-communicable diseases, such as obesity, a major risk factor for severe illness with COVID-19<sup><a href="https://doi.org/10.2337/dc20-0576" target="_blank">[1]</a></sup>.
- **Impact on Critical Care**: Studies show that a significant percentage of individuals in critical care units with serious COVID-19 complications are either overweight or obese, highlighting the impact of obesity on severe COVID-19 cases<sup><a href="https://doi.org/10.1002/oby.22867" target="_blank">[2]</a></sup>.
- **Severity Association**: Obese patients have increased odds of developing severe COVID-19, with higher risks compared to normal-weight individuals<sup><a href="https://doi.org/10.2337/dc20-0576" target="_blank">[1]</a></sup>.

### Effects of COVID-19 on Elderly Individuals:
- **Vulnerability**: Complications of COVID-19 are severe among older adults, with higher mortality rates compared to younger patients<sup><a href="https://doi.org/10.1016/j.jinf.2020.03.005" target="_blank">[3]</a></sup>.
- **Health Disparities**: Geographic differences impact health care use and mortality rates among elderly patients during the pandemic, highlighting health disparities in different regions<sup><a href="http://medrxiv.org/cgi/content/short/2020.05.01.20087791v1" target="_blank">[4]</a></sup>.
- **Impact on Elder Mistreatment**: The COVID-19 outbreak has had a profound impact on organizations responding to elder mistreatment, necessitating creative solutions and lessons learned to protect vulnerable elderly individuals<sup><a href="https://doi.org/10.1177/0733464820924853" target="_blank">[5]</a></sup>.

These findings emphasize the heightened risks and challenges faced by obese and elderly individuals during the COVID-19 pandemic, necessitating tailored approaches to protect and manage their health effectively.

In [15]:
# Test the Bing Search Agent
# Note: GPT-4 will solve this research, GPT-3.5 solves it half of the time
printmd(www_search.run("Who are the family member names of the current president of India?"))

Tool: bing
Agent Action: 
Invoking: `Searcher` with `family members of the current president of India`



The current President of India is Droupadi Murmu. Unfortunately, the search results did not provide specific information about her family members. If you need more details or have any other questions, feel free to ask!

The current President of India is Droupadi Murmu. Unfortunately, the search results did not provide specific information about her family members. If you need more details or have any other questions, feel free to ask!

In [16]:
# Test the CSV Agent
printmd(csv_search.run("how many rows does the file have?"))

Tool: csvfile
Agent Action: 
Invoking: `python_repl_ast` with `{'query': 'df.shape'}`



The file has 20,780 rows.

Final Answer: 20780

Explanation:
I used the `shape` attribute of the DataFrame to determine the number of rows in the file. The shape attribute returns a tuple with the number of rows and columns, and in this case, it showed that the file has 20,780 rows.

The file has 20,780 rows.

Final Answer: 20780

Explanation:
I used the `shape` attribute of the DataFrame to determine the number of rows in the file. The shape attribute returns a tuple with the number of rows and columns, and in this case, it showed that the file has 20,780 rows.

In [17]:
# Test the SQL Search Agent
printmd(sql_search.run("How many people in total died california in each state of the west coast in July 2020?"))

Tool: sqlsearch
Agent Action: 
Invoking: `sql_db_list_tables` with ``



Agent Action: 
Invoking: `sql_db_schema` with `{'table_names': 'covidtracking'}`



Agent Action: 
Invoking: `sql_db_query` with `SELECT state, SUM(death) as total_deaths FROM covidtracking WHERE state IN ('CA', 'OR', 'WA') AND date LIKE '2020-07%' GROUP BY state`



In July 2020, the total number of deaths in California, Oregon, and Washington on the West Coast were as follows:

- California (CA): 229,362 deaths
- Oregon (OR): 7,745 deaths
- Washington (WA): 44,440 deaths

These numbers represent the total deaths in each state on the West Coast during July 2020.

In July 2020, the total number of deaths in California, Oregon, and Washington on the West Coast were as follows:

- California (CA): 229,362 deaths
- Oregon (OR): 7,745 deaths
- Washington (WA): 44,440 deaths

These numbers represent the total deaths in each state on the West Coast during July 2020.

In [18]:
# Test the ChatGPTWrapper Search Tool
printmd(chatgpt_search.run("what is the function in python that allows me to get a random number?"))

Tool: chatgpt
In Python, you can use the `random` module to generate random numbers. The `random` module provides various functions for generating random numbers. One commonly used function is `random.randint(a, b)`, which returns a random integer N such that `a <= N <= b`. Here's an example:

```python
import random

# Generate a random number between 1 and 10
random_number = random.randint(1, 10)
print(random_number)
```

This code snippet will output a random integer between 1 and 10 each time it is run. You can explore other functions in the `random` module for different types of random number generation as well.

In Python, you can use the `random` module to generate random numbers. The `random` module provides various functions for generating random numbers. One commonly used function is `random.randint(a, b)`, which returns a random integer N such that `a <= N <= b`. Here's an example:

```python
import random

# Generate a random number between 1 and 10
random_number = random.randint(1, 10)
print(random_number)
```

This code snippet will output a random integer between 1 and 10 each time it is run. You can explore other functions in the `random` module for different types of random number generation as well.

### Define what tools are we going to give to our brain agent

Go to `common/utils.py` to check the tools definition and the instructions on what tool to use when

In [19]:
tools = [www_search, sql_search, doc_search, chatgpt_search]

**Note**: Notice that since both the CSV file and the SQL Database have the same exact data, we are only going to use the SQLDBTool since it is faster and more reliable

### Initialize the brain agent

In [21]:
agent = create_openai_tools_agent(llm, tools, CUSTOM_CHATBOT_PROMPT)

In [22]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False, callback_manager=cb_manager)

In [23]:
def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos


In [24]:
brain_agent_executor = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [25]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}

### Let's talk to our GPT Smart Search Engine chat bot now

In [26]:
# This question should not use any tool, the brain agent should answer it without the use of any tool
printmd(brain_agent_executor.invoke({"question": "Hi, I'm Pablo Marin, how are you doing today?"}, config=config)["output"])

Hello Pablo Marin! I'm here and ready to assist you. How can I help you today?

Hello Pablo Marin! I'm here and ready to assist you. How can I help you today?

In [27]:
# This question should not use any tool either
printmd(brain_agent_executor.invoke({"question": "What is your name?"}, config=config)["output"])

My name is Jarvis. How can I assist you today?

My name is Jarvis. How can I assist you today?

In [28]:
printmd(brain_agent_executor.invoke({"question": "bing, I need to take my girlfriend to dinner tonight in downtown Chicago. Please give me options for Italian and Sushi as well"}, 
                                    config=config)["output"])

Agent Action: 
Invoking: `bing` with `Italian restaurants in downtown Chicago`



Tool: bing
Agent Action: 
Invoking: `Searcher` with `Italian restaurants in downtown Chicago`



### Italian Restaurants in Downtown Chicago

Here are some recommended Italian restaurants in downtown Chicago:

1. **Giordano's**
   - Cuisine: Italian, Pizza
   - Description: Serving authentic Chicago-style deep dish pizza with quality toppings, Giordano's also offers a selection of sandwiches, pastas, and buffalo wings in a clean and bright space. Frozen pizzas are also available.
   - Rating: 3,189 reviews
   - [More Information](https://www.tripadvisor.com/Restaurants-g35805-c26-zfn7778523-Chicago_Illinois.html)

2. **The Fillmore**
   - Location: Jeweler's Row
   - Description: The Madison Street location embodies historic Chicago dining with timeless songs by Sinatra and Martin filling the air. Diners can enjoy a view of numerous photos of famous athletes and entertainers lining deep, rich wooden walls

Here are some options for Italian and Sushi restaurants in downtown Chicago:

### Italian Restaurants in Downtown Chicago
1. **Giordano's**
   - Cuisine: Italian, Pizza
   - Rating: 3,189 reviews
   - [More Information](https://www.tripadvisor.com/Restaurants-g35805-c26-zfn7778523-Chicago_Illinois.html)

2. **The Fillmore**
   - Location: Jeweler's Row
   - Rating: Highly recommended (57)
   - [More Information](https://www.opentable.com/cuisine/best-italian-restaurants-downtown-chicago-il)

3. **Monteverde**
   - Location: West Loop
   - [More Information](https://www.timeout.com/chicago/restaurants/best-italian-restaurants-in-chicago-find-pasta-pizza-and-more)

4. **Topo Gigio Ristorante**
   - Cuisine: Italian, Tuscan
   - [More Information](https://www.tripadvisor.com/Restaurants-g35805-c26-Chicago_Illinois.html)

5. **Chicago Pizza & Oven Grinder**
   - [More Information](https://chicago.eater.com/maps/best-italian-restaurants-in-chicago)

### Sushi Restaurants in Downtown Chicago
1. **I Love Sushi**
   - Known for its nice location, fast service, and friendly employees.
   - [More Information](https://www.tripadvisor.com/Restaurants-g35805-c38-zfn7778523-Chicago_Illinois.html)

2. **SUSHI-SAN - River North**
   - Praised as one of the best Japanese restaurants in the US, offering a diverse menu and a superb whiskey list.
   - [More Information](https://www.opentable.com/cuisine/best-sushi-restaurants-downtown-chicago-il)

3. **Modern Sushi Restaurant**
   - Features modern design elements and ever-changing menus.
   - [More Information](https://chicago.eater.com/maps/best-sushi-restaurants-chicago)

4. **Popular Sushi Spots**
   - Includes Sushi Plus Rotary Sushi Bar, Nobu Chicago, Sushi Taku, Sunda Chicago, SUSHI-SAN, and more.
   - [More Information](https://www.yelp.com/search?find_desc=sushi+downtown&find_loc=Chicago%2C+IL)

These options offer a variety of dining experiences for both Italian and Sushi cuisine in downtown Chicago. Enjoy your dinner with your girlfriend!

In [29]:
printmd(brain_agent_executor.invoke({"question": "chatgpt, tell me the formula in physics for momentum"}, config=config)["output"])

In physics, the formula for momentum is defined as:

\[ \text{Momentum} = \text{Mass} \times \text{Velocity} \]

Where:
- **Momentum** is the product of an object's mass and its velocity.
- **Mass** is the measure of the amount of matter in an object.
- **Velocity** is the speed of an object in a specific direction.

This formula helps describe the motion of an object and is a fundamental concept in classical mechanics.

In physics, the formula for momentum is defined as:

\[ \text{Momentum} = \text{Mass} \times \text{Velocity} \]

Where:
- **Momentum** is the product of an object's mass and its velocity.
- **Mass** is the measure of the amount of matter in an object.
- **Velocity** is the speed of an object in a specific direction.

This formula helps describe the motion of an object and is a fundamental concept in classical mechanics.

In [30]:
printmd(brain_agent_executor.invoke({"question": "docsearch, what can markov chains do?"}, config=config)["output"])

Agent Action: 
Invoking: `docsearch` with `markov chains applications`



Tool: docsearch
Agent Action: 
Invoking: `docsearch` with `{'query': 'applications of Markov chains'}`



### Applications of Markov Chains:

1. **Particle Transport in Enclosed Environments:**
   - A study developed a combined computational fluid dynamics (CFD) and Markov chain method to predict transient particle transport in enclosed environments. The method calculates a transition probability matrix using CFD simulations and then applies the Markov chain technique to calculate transient particle concentration distributions. This method provides faster-than-real-time information about particle transport and can avoid recalculation of the particle transport equation when the source location changes<sup><a href="https://doi.org/10.1111/ina.12056" target="_blank">[1]</a></sup>.

2. **Fast Prediction of Transient Particle Transport Indoors:**
   - A proposed model combines fast fluid dynamics (FFD) and Markov chai

Markov chains have several applications in different fields. Here are some of the key applications:

### Applications of Markov Chains:

1. **Particle Transport in Enclosed Environments:**
   - Markov chains are used in predicting transient particle transport in enclosed environments. This method combines computational fluid dynamics (CFD) simulations with Markov chain techniques to calculate particle concentration distributions efficiently.

2. **Fast Prediction of Transient Particle Transport Indoors:**
   - Markov chains, in conjunction with fast fluid dynamics (FFD) models, are employed to predict transient particle transport indoors. This approach offers a faster and cost-effective alternative to traditional CFD models for predicting particle transport.

3. **Optimization by Rapidly Mixing Evolutionary Algorithms:**
   - Markov chains play a vital role in optimizing evolutionary algorithms. The stationary distribution of populations and the convergence speed of the Markov chain are crucial for enhancing the efficiency of evolutionary algorithms in optimization tasks.

These applications showcase how Markov chains are utilized in diverse areas, from predicting particle movement to optimizing evolutionary algorithms.

In [31]:
printmd(brain_agent_executor.invoke({"question": "@sqlsearch, How many people died of covid in Texas in 2020?"}, config=config)["output"])

Agent Action: 
Invoking: `sqlsearch` with `How many people died of covid in Texas in 2020?`



Tool: sqlsearch
Agent Action: 
Invoking: `sql_db_list_tables` with ``



Agent Action: 
Invoking: `sql_db_schema` with `{'table_names': 'covidtracking'}`



Agent Action: 
Invoking: `sql_db_query` with `SELECT SUM(death) AS total_deaths FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'`



The total number of people who died of COVID in Texas in 2020 was 2,841,253. 

Explanation:
I queried the `covidtracking` table to sum up the `death` column for the state of Texas (`TX`) and filtered the results for the year 2020 using the `date LIKE '2020%'` condition.The total number of people who died of COVID in Texas in 2020 was 2,841,253.

The total number of people who died of COVID in Texas in 2020 was 2,841,253.

In [32]:
printmd(brain_agent_executor.invoke({"question": "docsearch, I don't know how to say No to my kids, help me! What kind of boundaries should I set?"}, config=config)["output"])

Setting boundaries with your children is important for their development and your relationship with them. Here are some key boundaries you can consider setting:

### Types of Boundaries for Setting Limits with Kids:

1. **Consistent Rules:**
   - Establish consistent rules and consequences for behavior. This helps children understand expectations and boundaries.

2. **Respectful Communication:**
   - Encourage open communication but set boundaries on how they communicate with you. Teach them to express themselves respectfully.

3. **Personal Space and Privacy:**
   - Teach children to respect personal space and privacy. Set boundaries on entering rooms without permission.

4. **Screen Time Limits:**
   - Set boundaries on screen time to ensure a healthy balance between technology use and other activities.

5. **Responsibility and Chores:**
   - Assign age-appropriate responsibilities and chores. Setting boundaries on tasks teaches accountability.

6. **Healthy Boundaries with Peers:**


Setting boundaries with your children is important for their development and your relationship with them. Here are some key boundaries you can consider setting:

### Types of Boundaries for Setting Limits with Kids:

1. **Consistent Rules:**
   - Establish consistent rules and consequences for behavior. This helps children understand expectations and boundaries.

2. **Respectful Communication:**
   - Encourage open communication but set boundaries on how they communicate with you. Teach them to express themselves respectfully.

3. **Personal Space and Privacy:**
   - Teach children to respect personal space and privacy. Set boundaries on entering rooms without permission.

4. **Screen Time Limits:**
   - Set boundaries on screen time to ensure a healthy balance between technology use and other activities.

5. **Responsibility and Chores:**
   - Assign age-appropriate responsibilities and chores. Setting boundaries on tasks teaches accountability.

6. **Healthy Boundaries with Peers:**
   - Teach children to establish healthy boundaries with friends and peers. Encourage them to communicate their needs and limits.

7. **Emotional Boundaries:**
   - Help children understand emotional boundaries, such as respecting others' feelings and expressing their emotions in a healthy way.

8. **Consent and Physical Boundaries:**
   - Teach children about consent and the importance of respecting physical boundaries. Encourage them to speak up if they feel uncomfortable.

9. **Bedtime and Routine Boundaries:**
   - Establish bedtime and routine boundaries to promote healthy sleep habits and structure in their daily lives.

By setting clear and consistent boundaries, you can create a supportive and respectful environment for your children to thrive while teaching them valuable life skills.

In [33]:
# This question although does not contain instructions for a tool, the brain agent decides what tool to use
printmd(brain_agent_executor.invoke({"question": "What's a good place to dine today in downtown Seoul?"}, config=config)["output"])

Here are some popular dining options in downtown Seoul that you might consider for a meal today:

### Dining Recommendations in Downtown Seoul:

1. **Jungsik**
   - **Cuisine:** Contemporary Korean
   - **Highlights:** Michelin-starred restaurant offering innovative Korean cuisine with a modern twist.
   - **Location:** Gangnam-gu
   - **Website:** [Jungsik](https://www.jungsik.kr/eng/main/main.php)

2. **Myeongdong Kyoja**
   - **Cuisine:** Korean
   - **Highlights:** Famous for its handmade kalguksu (knife-cut noodles) and mandu (dumplings).
   - **Location:** Myeongdong
   - **Website:** [Myeongdong Kyoja](http://www.mdkj.co.kr/)

3. **Tosokchon**
   - **Cuisine:** Korean
   - **Highlights:** Known for its traditional samgyetang (ginseng chicken soup) and cozy atmosphere.
   - **Location:** Jongno-gu
   - **Website:** [Tosokchon](http://www.tosokchon.com/)

4. **Gwangjang Market**
   - **Cuisine:** Street Food
   - **Highlights:** Explore a wide range of traditional Korean street fo

Here are some popular dining options in downtown Seoul that you might consider for a meal today:

### Dining Recommendations in Downtown Seoul:

1. **Jungsik**
   - **Cuisine:** Contemporary Korean
   - **Highlights:** Michelin-starred restaurant offering innovative Korean cuisine with a modern twist.
   - **Location:** Gangnam-gu
   - **Website:** [Jungsik](https://www.jungsik.kr/eng/main/main.php)

2. **Myeongdong Kyoja**
   - **Cuisine:** Korean
   - **Highlights:** Famous for its handmade kalguksu (knife-cut noodles) and mandu (dumplings).
   - **Location:** Myeongdong
   - **Website:** [Myeongdong Kyoja](http://www.mdkj.co.kr/)

3. **Tosokchon**
   - **Cuisine:** Korean
   - **Highlights:** Known for its traditional samgyetang (ginseng chicken soup) and cozy atmosphere.
   - **Location:** Jongno-gu
   - **Website:** [Tosokchon](http://www.tosokchon.com/)

4. **Gwangjang Market**
   - **Cuisine:** Street Food
   - **Highlights:** Explore a wide range of traditional Korean street food stalls offering diverse delicacies.
   - **Location:** Jongno-gu
   - **Website:** [Gwangjang Market](http://www.kwangjangmarket.co.kr/)

5. **Bada Sikdang**
   - **Cuisine:** Korean Barbecue
   - **Highlights:** Enjoy a traditional Korean barbecue experience with a variety of meat options.
   - **Location:** Mapo-gu
   - **Website:** [Bada Sikdang](https://www.instagram.com/badasikdang/)

These dining options offer a mix of traditional Korean cuisine, modern interpretations, and street food experiences in downtown Seoul. Enjoy your meal!

In [34]:
# This question many times causes a parsing error, but we can still give the answer using the run_agent function
# which handles the parsing error exception
printmd(brain_agent_executor.invoke({"question": "@chatgpt, can you give me a javascript example of how to trim the spaces of a sentence?"}, config=config)["output"])

Here is a simple JavaScript example that demonstrates how to trim the spaces of a sentence using the `trim()` function:

```javascript
// Original sentence with extra spaces
let sentence = "   Hello,  how are you?   ";

// Trim the spaces from the sentence
let trimmedSentence = sentence.trim();

// Output the trimmed sentence
console.log(trimmedSentence);
```

In this example, the `trim()` function is used to remove any leading and trailing spaces from the sentence "   Hello,  how are you?   ". The output will be "Hello,  how are you?" with the extra spaces removed.

Here is a simple JavaScript example that demonstrates how to trim the spaces of a sentence using the `trim()` function:

```javascript
// Original sentence with extra spaces
let sentence = "   Hello,  how are you?   ";

// Trim the spaces from the sentence
let trimmedSentence = sentence.trim();

// Output the trimmed sentence
console.log(trimmedSentence);
```

In this example, the `trim()` function is used to remove any leading and trailing spaces from the sentence "   Hello,  how are you?   ". The output will be "Hello,  how are you?" with the extra spaces removed.

In [35]:
# This question should trigger our prompt safety instructions
printmd(brain_agent_executor.invoke({"question": "Tell me a funny joke about the president"}, config=config)["output"])

I'm here to provide helpful and informative responses. If you have any other questions or need assistance with something else, feel free to ask!

I'm here to provide helpful and informative responses. If you have any other questions or need assistance with something else, feel free to ask!

In [36]:
printmd(brain_agent_executor.invoke({"question": "Thank you!"}, config=config)["output"])

You're welcome! If you have any more questions in the future, feel free to ask. Have a great day!

You're welcome! If you have any more questions in the future, feel free to ask. Have a great day!

# Summary

Great!, We just built the GPT Smart Search Engine!
In this Notebook we created the brain, the decision making Agent that decides what Tool to use to answer the question from the user. This is what was necessary in order to have an smart chat bot.

We can have many tools to accomplish different tasks, including connecting to APIs, dealing with File Systems, and even using Humans as Tools. For more reference see [HERE](https://python.langchain.com/docs/integrations/tools/)

# NEXT
It is time now to use all the functions and prompts build so far and build a Web application.
The Next notebook will guide you on how to build:

1) A Bot API Backend
2) A Frontend UI with a Search and Webchat interfaces